# Mornington Crescent Interactive Compiler
E Krupczak - 16 July 2018

## Mornington Crescent special stops

| Station | Special meaning| Description |
| :----|:--- |:---|
| Upminster | add | The accumulator’s value is set to the sum of ßits current value and the station’s current value, if both are integers (otherwise, standard swap).
|
| Chalfont & Latimer | multiply | The accumulator’s value is set to the product of its current value and the station’s current value, if both are integers (otherwise, standard swap).
|
| Cannon Street | integer division | The accumulator’s value is set to the quotient (rounded towards 0) of the station’s current value divided by the accumulator’s current value (or the empty string if the accumulator is 0) if both are integers (otherwise, standard swap).
|
| Preston Road | remainder | The accumulator’s value is set to the remainder of the station’s current value divided by the accumulator’s current value (or the empty string if the accumulator is 0) if both are integers (otherwise, standard swap).
|
| Bounds Green | max | The accumulator’s value is set to the station’s current value only if it is larger, if both are integers (otherwise, standard swap).
|
| Manor House | bitwise NOR (OR+NOT) | Just like the above arithmetic operators, but with bitwise operations. The shift operations only trigger if the shift amount is positive.
|
| Holland Park | bitwise AND
|
| Turnham Green | bitwise shift-right
|
| Stepney Green | bitwise shift-left
|
| Russell Square | square | The accumulator’s value is set to the square of the station’s current value if it is an integer (otherwise, standard swap).
|
| Notting Hill Gate | bitwise NOT | The accumulator’s value is set to the bitwise NOT of the station’s current value if it is an integer (otherwise, standard swap).
|
| Parsons Green | parse string to integer | If the accumulator contains a string, attempts finding and parsing an integer in it. The accumulator is set to the first integer found (or 0 if none), and the station’s value is set to the rest of the string (or <code>""</code>). (If the accumulator is an integer, standard swap occurs. Note that this condition is inconsistent with other instructions, which decide this based on their current value rather than the accumulator.) Example: If the accumulator value is <code>"a-2b4"</code>, the accumulator becomes <code>-2</code> and the station <code>"b4"</code>.
|
| Seven Sisters | 7 | This station’s value is always 7. The accumulator’s value is set to 7 and its previous value discarded.
|
| Charing Cross | character ↔ codepoint | If the station’s value is a non-empty string, the accumulator is set to the Unicode codepoint of the first character; if it’s the empty string, to 0; if it is an integer, the accumulator is set to the Unicode character with this codepoint.
|
| Paddington | string concatenation | The accumulator is set to the concatenation of the station’s value followed by the accumulator’s previous value, if both are strings (otherwise, standard swap).
|
| Gunnersbury | left substring | If one of the two values is a string ''s'' and the other an integer ''i'', the accumulator is set to the string containing the first ''i'' characters of ''s''. If ''i'' is negative or out of range, a runtime error occurs. If they are both strings or both integers, standard swap occurs.
|
| Mile End | right substring | Like Gunnersbury, but the accumulator is set to the '''last''' ''i'' characters.
|
| Upney | upper-case | Sets the accumulator to the upper-case of the station’s value if it is a string (otherwise, standard swap).
|
| Hounslow Central | lower-case | Sets the accumulator to the lower-case of the station’s value if it is a string (otherwise, standard swap).
|
| Turnpike Lane | reverse string | Sets the accumulator to the reverse of the station’s value if it is a string (otherwise, standard swap).
|
| Bank | | When a value is stored in Bank, Hammersmith is also set to the same value.
|
| Hammersmith | | Always retains its current value. The value of the accumulator is discarded and replaced.
|
| Temple | continuation | This station does not have a current value. The accumulator remains unaltered. Landing on this station pushes the current instruction pointer onto the jumpstack. 
|
| Angel | if | This station does not have a current value. The accumulator remains unaltered. If the accumulator is the integer 0, nothing happens. Otherwise, execution continues at the instruction stored at the top of the jumpstack (without popping it) and the data pointer is teleported to Temple.
|
| Marble Arch | pop | This station does not have a current value. The accumulator remains unaltered. Landing on this station pops the topmost value off the jumpstack. Nothing else happens.
|
| Mornington Crescent | output/exit | The value of the accumulator is printed and the program is terminated.
|


## Mornington Crescent Station List
First we make a dictionary connecting stations to their lines, so we can tell if an instruction set is valid. We will borrow this one from github user padarom's esoterpret repository. 

In [1]:
# %load defaults.py
"""
Default values for Mornington Crescent
From https://github.com/padarom/esoterpret
Forked on 16 July 2018 by E. Krupczak

"""
import re
from collections import defaultdict

_StationString = """Acton Town [Piccadilly] [District]
Aldgate [Metropolitan] [Circle]
Aldgate East [Hammersmith & City] [District]
Alperton [Piccadilly]
Amersham [Metropolitan]
Angel [Northern]
Archway [Northern]
Arnos Grove [Piccadilly]
Arsenal [Piccadilly]
Baker Street [Hammersmith & City] [Circle] [Metropolitan] [Bakerloo] [Jubilee]
Balham [Northern]
Bank [Central] [Waterloo & City] [Northern] [District] [Circle]
Barbican [Hammersmith & City] [Circle] [Metropolitan]
Barking [Hammersmith & City] [District]
Barkingside [Central]
Barons Court [Piccadilly] [District]
Bayswater [Circle] [District]
Becontree [District]
Belsize Park [Northern]
Bermondsey [Jubilee]
Bethnal Green [Central]
Blackfriars [Circle] [District]
Blackhorse Road [Victoria]
Bond Street [Jubilee] [Central]
Borough [Northern]
Boston Manor [Piccadilly]
Bounds Green [Piccadilly]
Bow Road [Hammersmith & City] [District]
Brent Cross [Northern]
Brixton [Victoria]
Bromley-by-Bow [Hammersmith & City] [District]
Buckhurst Hill [Central]
Burnt Oak [Northern]
Caledonian Road [Piccadilly]
Camden Town [Northern]
Canada Water [Jubilee]
Canary Wharf [Jubilee]
Canning Town [Jubilee]
Cannon Street [Circle] [District]
Canons Park [Jubilee]
Chalfont & Latimer [Metropolitan]
Chalk Farm [Northern]
Chancery Lane [Central]
Charing Cross [Bakerloo] [Northern]
Chesham [Metropolitan]
Chigwell [Central]
Chiswick Park [District]
Chorleywood [Metropolitan]
Clapham Common [Northern]
Clapham North [Northern]
Clapham South [Northern]
Cockfosters [Piccadilly]
Colindale [Northern]
Colliers Wood [Northern]
Covent Garden [Piccadilly]
Croxley [Metropolitan]
Dagenham East [District]
Dagenham Heathway [District]
Debden [Central]
Dollis Hill [Jubilee]
Ealing Broadway [Central] [District]
Ealing Common [Piccadilly] [District]
Earl's Court [District] [Piccadilly]
East Acton [Central]
Eastcote [Metropolitan] [Piccadilly]
East Finchley [Northern]
East Ham [Hammersmith & City] [District]
East Putney [District]
Edgware [Northern]
Edgware Road [Hammersmith & City] [Circle] [District]
Edgware Road [Bakerloo]
Elephant & Castle [Bakerloo] [Northern]
Elm Park [District]
Embankment [Northern] [Bakerloo] [Circle] [District]
Epping [Central]
Euston [Northern] [Victoria]
Euston Square [Hammersmith & City] [Circle] [Metropolitan]
Fairlop [Central]
Farringdon [Hammersmith & City] [Circle] [Metropolitan]
Finchley Central [Northern]
Finchley Road [Metropolitan] [Jubilee]
Finsbury Park [Piccadilly] [Victoria]
Fulham Broadway [District]
Gants Hill [Central]
Gloucester Road [Piccadilly] [Circle] [District]
Golders Green [Northern]
Goldhawk Road [Hammersmith & City] [Circle]
Goodge Street [Northern]
Grange Hill [Central]
Great Portland Street [Hammersmith & City] [Circle] [Metropolitan]
Greenford [Central]
Green Park [Jubilee] [Victoria] [Piccadilly]
Gunnersbury [District]
Hainault [Central]
Hammersmith [Piccadilly] [District]
Hammersmith [Hammersmith & City] [Circle]
Hampstead [Northern]
Hanger Lane [Central]
Harlesden [Bakerloo]
Harrow & Wealdstone [Bakerloo]
Harrow-on-the-Hill [Metropolitan]
Hatton Cross [Piccadilly]
Heathrow Terminal 4 [Piccadilly]
Heathrow Terminal 5 [Piccadilly]
Heathrow Terminals 1, 2, 3 [Piccadilly]
Hendon Central [Northern]
High Barnet [Northern]
Highbury & Islington [Victoria]
Highgate [Northern]
High Street Kensington [Circle] [District]
Hillingdon [Metropolitan] [Piccadilly]
Holborn [Piccadilly] [Central]
Holland Park [Central]
Holloway Road [Piccadilly]
Hornchurch [District]
Hounslow Central [Piccadilly]
Hounslow East [Piccadilly]
Hounslow West [Piccadilly]
Hyde Park Corner [Piccadilly]
Ickenham [Metropolitan] [Piccadilly]
Kennington [Northern]
Kensal Green [Bakerloo]
Kensington [District]
Kentish Town [Northern]
Kenton [Bakerloo]
Kew Gardens [District]
Kilburn [Jubilee]
Kilburn Park [Bakerloo]
Kingsbury [Jubilee]
King's Cross St. Pancras [Victoria] [Piccadilly] [Northern] [Circle] [Hammersmith & City] [Metropolitan]
Knightsbridge [Piccadilly]
Ladbroke Grove [Hammersmith & City] [Circle]
Lambeth North [Bakerloo]
Lancaster Gate [Central]
Latimer Road [Hammersmith & City] [Circle]
Leicester Square [Northern] [Piccadilly]
Leyton [Central]
Leytonstone [Central]
Liverpool Street [Circle] [Hammersmith & City] [Metropolitan] [Central]
London Bridge [Northern] [Jubilee]
Loughton [Central]
Maida Vale [Bakerloo]
Manor House [Piccadilly]
Mansion House [Circle] [District]
Marble Arch [Central]
Marylebone [Bakerloo]
Mile End [Central] [Hammersmith & City] [District]
Mill Hill East [Northern]
Moorgate [Northern] [Hammersmith & City] [Circle] [Metropolitan]
Moor Park [Metropolitan]
Morden [Northern]
Mornington Crescent [Northern]
Neasden [Jubilee]
Newbury Park [Central]
North Acton [Central]
North Ealing [Piccadilly]
Northfields [Piccadilly]
North Greenwich [Jubilee]
North Harrow [Metropolitan]
Northolt [Central]
North Wembley [Bakerloo]
Northwick Park [Metropolitan]
Northwood [Metropolitan]
Northwood Hills [Metropolitan]
Notting Hill Gate [Circle] [District] [Central]
Oakwood [Piccadilly]
Old Street [Northern]
Osterley [Piccadilly]
Oval [Northern]
Oxford Circus [Bakerloo] [Victoria] [Central]
Paddington [Bakerloo] [Circle] [District]
Paddington [Circle] [Hammersmith & City]
Park Royal [Piccadilly]
Parsons Green [District]
Perivale [Central]
Piccadilly Circus [Bakerloo] [Piccadilly]
Pimlico [Victoria]
Pinner [Metropolitan]
Plaistow [Hammersmith & City] [District]
Preston Road [Metropolitan]
Putney Bridge [District]
Queensbury [Jubilee]
Queen's Park [Bakerloo]
Queensway [Central]
Ravenscourt Park [District]
Rayners Lane [Metropolitan] [Piccadilly]
Redbridge [Central]
Regent's Park [Bakerloo]
Richmond [District]
Rickmansworth [Metropolitan]
Roding Valley [Central]
Royal Oak [Circle] [Hammersmith & City]
Ruislip [Metropolitan] [Piccadilly]
Ruislip Gardens [Central]
Ruislip Manor [Metropolitan] [Piccadilly]
Russell Square [Piccadilly]
Seven Sisters [Victoria]
Shepherd's Bush [Central]
Shepherd's Bush Market [Circle] [Hammersmith & City]
Sloane Square [Circle] [District]
Snaresbrook [Central]
South Ealing [Piccadilly]
Southfields [District]
Southgate [Piccadilly]
South Harrow [Piccadilly]
South Kensington [Piccadilly] [Circle] [District]
South Kenton [Bakerloo]
South Ruislip [Central]
Southwark [Jubilee]
South Wimbledon [Northern]
South Woodford [Central]
Stamford Brook [District]
Stanmore [Jubilee]
Stepney Green [Hammersmith & City] [District]
St. James's Park [Circle] [District]
St. John's Wood [Jubilee]
Stockwell [Victoria] [Northern]
Stonebridge Park [Bakerloo]
St. Paul's [Central]
Stratford [Central]  [Jubilee]
Sudbury Hill [Piccadilly]
Sudbury Town [Piccadilly]
Swiss Cottage [Jubilee]
Temple [Circle] [District]
Theydon Bois [Central]
Tooting Bec [Northern]
Tooting Broadway [Northern]
Tottenham Court Road [Northern] [Central]
Tottenham Hale [Victoria]
Totteridge & Whetstone [Northern]
Tower Hill [Circle] [District]
Tufnell Park [Northern]
Turnham Green [District]
Turnpike Lane [Piccadilly]
Upminster [District]
Upminster Bridge [District]
Upney [District]
Upton Park [Hammersmith & City] [District]
Uxbridge [Metropolitan] [Piccadilly]
Vauxhall [Victoria]
Victoria [Victoria] [Circle] [District]
Walthamstow Central [Victoria]
Wanstead [Central]
Warren Street [Northern] [Victoria]
Warwick Avenue [Bakerloo]
Waterloo [Bakerloo] [Northern] [Waterloo & City] [Jubilee]
Watford [Metropolitan]
Wembley Central [Bakerloo]
Wembley Park [Metropolitan] [Jubilee]
West Acton [Central]
Westbourne Park [Circle] [Hammersmith & City]
West Brompton [District]
West Finchley [Northern]
West Ham [Jubilee] [Hammersmith & City] [District]
West Hampstead [Jubilee]
West Harrow [Metropolitan]
West Kensington [District]
Westminster [Circle] [District] [Jubilee]
West Ruislip [Central]
Whitechapel [Hammersmith & City] [District]
White City [Central]
Willesden Green [Jubilee]
Willesden Junction [Bakerloo]
Wimbledon [District]
Wimbledon Park [District]
Woodford [Central]
Wood Green [Piccadilly]
Wood Lane [Circle] [Hammersmith & City]
Woodside Park [Northern]"""

_matches = map(lambda line: re.search("^([^\[\]]*?)\s*((\[[^\[\]]*\]\s*)*)$", line), _StationString.split("\n"))

Stations = defaultdict(list)
Lines    = set()

for match in _matches:
    for line in re.compile("\[([^\[\]]*)\]").findall(match.group(2)):
        Stations[match.group(1)].append(line)
        Lines.add(line)


In [2]:
display(Stations)

defaultdict(list,
            {'Acton Town': ['Piccadilly', 'District'],
             'Aldgate': ['Metropolitan', 'Circle'],
             'Aldgate East': ['Hammersmith & City', 'District'],
             'Alperton': ['Piccadilly'],
             'Amersham': ['Metropolitan'],
             'Angel': ['Northern'],
             'Archway': ['Northern'],
             'Arnos Grove': ['Piccadilly'],
             'Arsenal': ['Piccadilly'],
             'Baker Street': ['Hammersmith & City',
              'Circle',
              'Metropolitan',
              'Bakerloo',
              'Jubilee'],
             'Balham': ['Northern'],
             'Bank': ['Central',
              'Waterloo & City',
              'Northern',
              'District',
              'Circle'],
             'Barbican': ['Hammersmith & City', 'Circle', 'Metropolitan'],
             'Barking': ['Hammersmith & City', 'District'],
             'Barkingside': ['Central'],
             'Barons Court': ['Piccadilly', 'District'

## 'Hello World' in Mornington Crescent
We will load a hello world program in to use as our sample program

In [ ]:
# %load helloworld.mcresc
Take Northern Line to Hendon Central
Take Northern Line to Bank
Take Northern Line to Bank
Take District Line to Gunnersbury
Take District Line to Victoria
Take Victoria Line to Seven Sisters
Take Victoria Line to Victoria
Take Victoria Line to Victoria
Take District Line to Bank
Take District Line to Hammersmith
Take District Line to Cannon Street
Take District Line to Hammersmith
Take District Line to Cannon Street
Take District Line to Bank
Take District Line to Hammersmith
Take District Line to Upminster
Take District Line to Hammersmith
Take District Line to Upminster
Take District Line to Gunnersbury
Take District Line to Paddington
Take District Line to Acton Town
Take Piccadilly Line to Holloway Road
Take Piccadilly Line to Acton Town
Take Piccadilly Line to Acton Town
Take District Line to Gunnersbury
Take District Line to Hammersmith
Take District Line to Notting Hill Gate
Take District Line to Upminster
Take District Line to Notting Hill Gate
Take District Line to Upminster
Take District Line to Victoria
Take Victoria Line to Seven Sisters
Take Victoria Line to Victoria
Take Victoria Line to Victoria
Take District Line to Upminster
Take District Line to Gunnersbury
Take District Line to Mile End
Take District Line to Hammersmith
Take District Line to Notting Hill Gate
Take District Line to Upminster
Take District Line to Upminster
Take District Line to Mile End
Take District Line to Paddington
Take District Line to Paddington
Take District Line to Acton Town
Take Piccadilly Line to Heathrow Terminals 1, 2, 3
Take Piccadilly Line to Holborn
Take Central Line to Holborn
Take Central Line to Mile End
Take District Line to Upminster
Take District Line to Hammersmith
Take District Line to Upminster
Take District Line to Barking
Take District Line to Hammersmith
Take District Line to Upminster
Take District Line to Gunnersbury
Take District Line to Barking
Take District Line to Gunnersbury
Take District Line to Paddington
Take District Line to Paddington
Take Circle Line to Wood Lane
Take Circle Line to Victoria
Take Circle Line to Victoria
Take District Line to Gunnersbury
Take District Line to Hammersmith
Take District Line to Upminster
Take District Line to Gunnersbury
Take District Line to Paddington
Take District Line to Paddington
Take District Line to Mile End
Take Central Line to Fairlop
Take Central Line to Mile End
Take District Line to Barking
Take District Line to Upminster
Take District Line to Upminster
Take District Line to Hammersmith
Take District Line to Notting Hill Gate
Take District Line to Upminster
Take District Line to Mile End
Take District Line to Gunnersbury
Take District Line to Paddington
Take District Line to Paddington
Take District Line to Hammersmith
Take District Line to Mile End
Take District Line to Richmond
Take District Line to Mile End
Take District Line to Paddington
Take District Line to Paddington
Take District Line to Richmond
Take District Line to Bank
Take District Line to Hammersmith
Take District Line to Upminster
Take District Line to Stepney Green
Take District Line to Hammersmith
Take District Line to Stepney Green
Take District Line to Upney
Take District Line to Notting Hill Gate
Take District Line to Notting Hill Gate
Take District Line to Notting Hill Gate
Take District Line to Upminster
Take District Line to Upney
Take District Line to Upminster
Take District Line to Bank
Take Circle Line to Bank
Take Northern Line to Charing Cross
Take Bakerloo Line to Charing Cross
Take Bakerloo Line to Paddington
Take Circle Line to Bank
Take Circle Line to Bank
Take Northern Line to Mornington Crescent

## Interpreter
Now we load in the program. I'm going to use pandas for this because I like pandas and this is my project - sorry!

### Load Mornington Crescent File and Check Validity

In [3]:
import pandas as pd
import re
program = pd.DataFrame(columns = ['Instruction', 'Station', 'Line', 'Accumulator', 'DataPointer', 'Jumpstack'])
program['Instruction'] = pd.read_table('helloworld.mcresc', header = None)[0]

#Error line as a test
#program.loc[10, 'Instruction'] = 'Take Error Line to Random Station'
#program.loc[0, 'Instruction'] = 'Take Circle Line to Bank'
#program.loc[len(program)-1, 'Instruction'] = 'Take Circle Line to Paddington'

#Simple regex to get station name and line name
#This is not robust and is easy to trick
program['Line'] = program.Instruction.apply(lambda x: re.findall('(?:Take\s)([a-zA-Z\s]+)(?:\sLine\sto\s)', x)[0])
program['Station'] = program.Instruction.apply(lambda x: re.findall('(?:\sto\s)([a-zA-Z0-9,&\s]+)', x)[0])

In [4]:
#Check that all stations are valid and are on their corresponding line
def station_check(line):
    #print('Station', line[1], '\nLine', line[2], '\nLines to station', Stations[line[1]])
    if line[1] in Stations.keys(): 
        #print('Valid station')
        if line[2] in Stations[line[1]]: 
            #print('Valid line')
            return True
    else: return False
program['ValidCheck'] = program.apply(station_check, axis=1)
if len(program[program.ValidCheck == False]) > 0:
    print('ERROR: The following lines are invalid:')
    display(program[program.ValidCheck == False])
    raise Exception

In [5]:
# Check that the program starts at Mornington Crescent 
if program.loc[0, 'Line'] not in Stations['Mornington Crescent']:
    raise Exception('Must begin at Mornington Crescent')
# Check that the program ends at Mornington Crescent
if program.loc[len(program)-1, 'Station'] != 'Mornington Crescent':
    raise Exception('Must end at Mornington Crescent')

In [6]:
# Make a dict to store the current values of each station
StationValues =dict(zip(list(Stations.keys()), list(Stations.keys())))
#Amend exceptions
StationValues['Seven Sisters'] = 7
StationValues['Angel'] = None
StationValues['Marble Arch'] = None

In [7]:
# Execute each instruction
import math
def standard_swap(row):
    StationValues[row.Station], row.Accumulator = row.Accumulator, StationValues[row.Station]
def execute(row):
    if row.Station == 'Upmister':
        if type(row.Accumulator) == int and type(StationValues[row.Station])==int :
            row.Accumulator += StationValues[row.Station]
        else: standard_swap(row)
    if row.Station == 'Chalfont & Latimer':
        if type(row.Accumulator) == int and type(StationValues[row.Station])==int :
            row.Accumulator *= StationValues[row.Station]
        else: standard_swap(row)
    if row.Station == 'Cannon Street':
        if type(row.Accumulator) == int and type(StationValues[row.Station])==int :
            if row.Accumulator == 0: row.Accumulator = ''
            else: row.Accumulator = math.floor(StationValues[row.Station]/row.Accumulator)
        else: standard_swap(row)

    else: standard_swap(row)

In [8]:
for index, row in program.iterrows():
    row['DataPointer'] = row['Station']
    execute(row)